In [1]:
import matplotlib.pyplot as plt
from IPython.display import display, Markdown, Latex
import numpy as np
import json
import pandas as pd
from scipy import stats
import os

In [2]:
experiments = [
            {'id': '1', 'source':'imdb', 'target':'uwcse', 'predicate':'workedunder', 'to_predicate':'advisedby', 'arity': 2},
            #{'id': '2', 'source':'uwcse', 'target':'imdb', 'predicate':'advisedby', 'to_predicate':'workedunder', 'arity': 2},
            {'id': '3', 'source':'imdb', 'target':'cora', 'predicate':'workedunder', 'to_predicate':'samevenue', 'arity': 2},
            {'id': '4', 'source':'cora', 'target':'imdb', 'predicate':'samevenue', 'to_predicate':'workedunder', 'arity': 2},
            ##{'id': '5', 'source':'uwcse', 'target':'cora', 'predicate':'advisedby', 'to_predicate':'samevenue', 'arity': 2},
            ##{'id': '6', 'source':'cora', 'target':'uwcse', 'predicate':'samevenue', 'to_predicate':'advisedby', 'arity': 2},
            {'id': '7', 'source':'yeast', 'target':'twitter', 'predicate':'proteinclass', 'to_predicate':'accounttype', 'arity': 2},
            {'id': '8', 'source':'twitter', 'target':'yeast', 'predicate':'accounttype', 'to_predicate':'proteinclass', 'arity': 2},
            {'id': '9', 'source':'nell_sports', 'target':'nell_finances', 'predicate':'teamplayssport', 'to_predicate':'companyeconomicsector', 'arity': 2},
            #{'id': '10', 'source':'nell_finances', 'target':'nell_sports', 'predicate':'companyeconomicsector', 'to_predicate':'teamplayssport', 'arity': 2}
]

In [3]:
def load_data(experiments, metric, treeboostler=False, rdnb=False):
    data = {}
    for item in experiments:
        experiment_title = item['id'] + '_' + item['source'] + '_' + item['target']
        
        if(treeboostler):
            file_path = os.path.split(os.getcwd())[0] + '/TreeBoostler/experiments-transfer/' + experiment_title + '/' + experiment_title + '.json'
        elif(rdnb):
            file_path = os.getcwd() + '/rdn-b-transfer-experiment/' + experiment_title + '/rdnb.json'
        else:
            file_path = os.getcwd() + '/experiments-transfer-k-1/' + experiment_title + '/' + experiment_title + '_fasttext_{}.json'.format(metric)
        
        #if(os.path.isfile(file_path)):
        with open(file_path, 'r') as fp:
            results = json.load(fp)

            data[experiment_title] = results
    return data

def load_data_no_rev(experiments, metric, treeboostler=False, rdnb=False):
    data = {}
    for item in experiments:
        experiment_title = item['id'] + '_' + item['source'] + '_' + item['target']
        
        if(treeboostler):
            file_path = os.path.split(os.getcwd())[0] + '/TreeBoostler/experiments-transfer/' + experiment_title + '/' + experiment_title + '.json'
        elif(rdnb):
            file_path = os.getcwd() + '/rdn-b-transfer-experiment/' + experiment_title + '/rdnb.json'
        else:
            file_path = os.getcwd() + '/experiments-no-revision/' + experiment_title + '/' + experiment_title + '_fasttext_{}.json'.format(metric)
        
        #if(os.path.isfile(file_path)):
        with open(file_path, 'r') as fp:
            results = json.load(fp)

            data[experiment_title] = results
    return data

In [12]:
data_softcosine = load_data(experiments, 'softcosine')
data_euclidean = load_data(experiments, 'euclidean')
data_wmd = load_data(experiments, 'wmd')

data_softcosine_no_rev = load_data_no_rev(experiments, 'softcosine')
data_euclidean_no_rev = load_data_no_rev(experiments, 'euclidean')
data_wmd_no_rev = load_data_no_rev(experiments, 'wmd')

treeboostler_data = load_data(experiments, '', True)

rdnb_data = load_data(experiments, '', False, True)

for j in range(len(experiments)):
    dataset = experiments[j]['id'] + '_' + experiments[j]['source'] + '_' + experiments[j]['target']
    #if dataset in data['results']: 
    display(Markdown('# Results for ' + dataset))
    table = []
    for metric in ['AUC ROC', 'AUC PR', 'CLL']: #, 'Learning and Revision time', 'Inference time']:
        display(Markdown('## ' + metric))
        
        trans_softcosine = np.zeros(len(data_softcosine[str(dataset)][0]))
        trans_softcosine_no_rev = np.zeros(len(data_softcosine_no_rev[str(dataset)][0]))
        
        trans_euclidean = np.zeros(len(data_euclidean[str(dataset)][0]))
        trans_euclidean_no_rev = np.zeros(len(data_euclidean_no_rev[str(dataset)][0]))
        
        trans_wmd = np.zeros(len(data_wmd[str(dataset)][0]))
        trans_wmd_no_rev = np.zeros(len(data_wmd_no_rev[str(dataset)][0]))
        
        tree_no_rev = np.zeros(len(treeboostler_data[str(dataset)][0]))
        tree = np.zeros(len(treeboostler_data[str(dataset)][0]))
        
        rdnb_no_rev = np.zeros(len(rdnb_data[str(dataset)][0]))
        rdnb = np.zeros(len(rdnb_data[str(dataset)][0]))
        
        #for m in range(len(treeboostler_data[str(dataset)][0])):
        m = 0
        trans_softcosine += np.array([item['transfer'][metric] for item in data_softcosine[str(dataset)][m]])
        trans_softcosine_no_rev += np.array([item['transfer'][metric] for item in data_softcosine_no_rev[str(dataset)][m]])

        trans_euclidean += np.array([item['transfer'][metric] for item in data_euclidean[str(dataset)][m]])
        trans_euclidean_no_rev += np.array([item['transfer'][metric] for item in data_euclidean_no_rev[str(dataset)][m]])

        trans_wmd += np.array([item['transfer'][metric] for item in data_wmd[str(dataset)][m]])
        trans_wmd_no_rev += np.array([item['transfer'][metric] for item in data_wmd_no_rev[str(dataset)][m]])

        tree_no_rev += np.array([item['transfer']['parameter'][metric] for item in treeboostler_data[str(dataset)][m]])
        tree += np.array([item['transfer'][metric] for item in treeboostler_data[str(dataset)][m]])
        
        # Fora do FOR

        rdnb_no_rev += np.array([item['rdn-b'][metric] for item in rdnb_data[str(dataset)][0]]) 
        rdnb += np.array([item['rdn-b'][metric] for item in rdnb_data[str(dataset)][0]])
            
        #trans_softcosine /= len(data_softcosine[str(dataset)])
        #trans_softcosine_no_rev /= len(data_softcosine_no_rev[str(dataset)])
        
        #trans_euclidean /= len(data_euclidean[str(dataset)])
        #trans_euclidean_no_rev /= len(data_euclidean_no_rev[str(dataset)])
        
        #trans_wmd /= len(data_wmd[str(dataset)])
        #trans_wmd_no_rev /= len(data_wmd_no_rev[str(dataset)])
        
        #tree /= len(treeboostler_data[str(dataset)])
        #tree_no_rev /= len(treeboostler_data[str(dataset)])
        
        #rdnb /= len(rdnb_data[str(dataset)])
        #rdnb_no_rev /= len(rdnb_data[str(dataset)])
        
        pvalue_table = []
        tvalue, pvalue = stats.ttest_rel(trans_softcosine,tree)
        tvalue2, pvalue2 = stats.ttest_rel(trans_softcosine, rdnb)
        pvalue_table.append(['TransBoostler Soft Cosine', '%.3f' % (pvalue), '%.3f' % (pvalue2)])
        del pvalue,pvalue2
        
        tvalue, pvalue = stats.ttest_rel(trans_euclidean,tree)
        tvalue2, pvalue2 = stats.ttest_rel(trans_euclidean,rdnb)

        pvalue_table.append(['TransBoostler Euclidean', '%.3f' % (pvalue), '%.3f' % (pvalue2)])
        del pvalue,pvalue2
        
        tvalue, pvalue = stats.ttest_rel(trans_wmd,tree)
        tvalue2, pvalue2 = stats.ttest_rel(trans_wmd,rdnb)
        
        pvalue_table.append(['TransBoostler WMD', '%.3f' % (pvalue), '%.3f' % (pvalue2)])
        del pvalue,pvalue2
        
        # No Revision
        
        tvalue, pvalue = stats.ttest_rel(trans_softcosine_no_rev,tree_no_rev)
        tvalue2, pvalue2 = stats.ttest_rel(trans_softcosine_no_rev,rdnb_no_rev)
        
        pvalue_table.append(['TransBoostler* Soft Cosine', '%.3f' % (pvalue), '%.3f' % (pvalue2)])
        del pvalue,pvalue2
        
        tvalue, pvalue = stats.ttest_rel(trans_euclidean_no_rev,tree_no_rev)
        tavlue2, pvalue2 = stats.ttest_rel(trans_euclidean_no_rev,rdnb_no_rev)
        
        pvalue_table.append(['TransBoostler* Euclidean', '%.3f' % (pvalue), '%.3f' % (pvalue2)])
        del pvalue,pvalue2
        
        tvalue, pvalue = stats.ttest_rel(trans_wmd_no_rev,tree_no_rev)
        tvalue, pvalue2 = stats.ttest_rel(trans_wmd_no_rev,rdnb_no_rev)
        
        pvalue_table.append(['TransBoostler* WMD', '%.3f' % (pvalue), '%.3f' % (pvalue2)])
        del pvalue,pvalue2
        
        display(pd.DataFrame(pvalue_table, columns=['p-value', 'TreeBoostler', 'RDN-B']))
        print('TransBoostler Soft Cosine', ','.join(str(v) for v in trans_softcosine))
        
        print('TransBoostler Euclidean', ','.join(str(v) for v in trans_euclidean))
        
        print('TransBoostler WMD', ','.join(str(v) for v in trans_wmd))
        
        print('TreeBoostler', ','.join(str(v) for v in tree))
        
        print('TransBoostler* Soft Cosine', ','.join(str(v) for v in trans_softcosine_no_rev))
        print('TransBoostler* Euclidean', ','.join(str(v) for v in trans_euclidean_no_rev))
        print('TransBoostler* WMD', ','.join(str(v) for v in trans_wmd_no_rev))
        
        print('TreeBoostler*', ','.join(str(v) for v in tree_no_rev))
        
        print('RDN-B', ','.join(str(v) for v in rdnb))
       

# Results for 1_imdb_uwcse

## AUC ROC

,p-value,TreeBoostler,RDN-B
0,TransBoostler Soft Cosine,0.481,0.234
1,TransBoostler Euclidean,0.805,0.384
2,TransBoostler WMD,0.508,0.135
3,TransBoostler* Soft Cosine,0.000,0.000
4,TransBoostler* Euclidean,0.016,0.014
5,TransBoostler* WMD,0.016,0.014


TransBoostler Soft Cosine 0.941863,0.929012,0.907846,0.947827,0.948144
TransBoostler Euclidean 0.94576,0.941423,0.908527,0.947708,0.947766
TransBoostler WMD 0.94807,0.934509,0.903786,0.946592,0.947854
TreeBoostler 0.949915,0.950746,0.905714,0.936897,0.952544
TransBoostler* Soft Cosine 0.609051,0.608882,0.608826,0.608066,0.607124
TransBoostler* Euclidean 0.908168,0.903558,0.904213,0.907372,0.905865
TransBoostler* WMD 0.908168,0.903558,0.904213,0.907372,0.905865
TreeBoostler* 0.935982,0.932863,0.904213,0.936897,0.937621
RDN-B 0.953654,0.94523,0.90564,0.951344,0.945339


## AUC PR

...

,p-value,TreeBoostler,RDN-B
0,TransBoostler Soft Cosine,0.949,0.629
1,TransBoostler Euclidean,0.529,0.836
2,TransBoostler WMD,0.245,0.567
3,TransBoostler* Soft Cosine,0.049,0.039
4,TransBoostler* Euclidean,0.049,0.039
5,TransBoostler* WMD,0.049,0.039


TransBoostler Soft Cosine 0.05731,0.102454,0.06435
TransBoostler Euclidean 0.073789,0.065684,0.057787
TransBoostler WMD 0.092757,0.086173,0.055069
TreeBoostler 0.089768,0.08552,0.045175
TransBoostler* Soft Cosine 0.001485,0.00151,0.001493
TransBoostler* Euclidean 0.001612,0.001568,0.001571
TransBoostler* WMD 0.001537,0.001527,0.001511
TreeBoostler* 0.089768,0.072937,0.038487
RDN-B 0.052288,0.047319,0.086231


## CLL

,p-value,TreeBoostler,RDN-B
0,TransBoostler Soft Cosine,0.003,0.709
1,TransBoostler Euclidean,0.002,0.620
2,TransBoostler WMD,0.002,0.035
3,TransBoostler* Soft Cosine,0.000,0.002
4,TransBoostler* Euclidean,0.001,0.019
5,TransBoostler* WMD,0.000,0.006


TransBoostler Soft Cosine -0.3353,-0.309711,-0.33635
TransBoostler Euclidean -0.329784,-0.314689,-0.337021
TransBoostler WMD -0.296252,-0.307588,-0.290245
TreeBoostler -0.16904,-0.164815,-0.162882
TransBoostler* Soft Cosine -0.368417,-0.366338,-0.371307
TransBoostler* Euclidean -0.375058,-0.358214,-0.365581
TransBoostler* WMD -0.363947,-0.372602,-0.360595
TreeBoostler* -0.16904,-0.172761,-0.164763
RDN-B -0.320674,-0.324576,-0.323881
